In [5]:
import ee
import pandas as pd
import geemap
import json

In [2]:
ee.Authenticate()

In [3]:
ee.Initialize(project='maps-disaster')

In [163]:
gaul1 = ee.FeatureCollection('FAO/GAUL/2015/level1')

# Filter for South Sulawesi in Indonesia
south_sulawesi = gaul1.filter(
    ee.Filter.And(
        ee.Filter.eq('ADM0_NAME', 'Indonesia'),
        ee.Filter.eq('ADM1_NAME', 'Sulawesi Selatan')
    )
)


In [164]:
# Load the MODIS Global Flood Database
flood_events = ee.ImageCollection("GLOBAL_FLOOD_DB/MODIS_EVENTS/V1") \
    .filterBounds(south_sulawesi)

# Get the first image in the collection to display (as an example)
# You might want to filter by date for a specific event
first_flood_event = flood_events.first()

# Define visualization parameters for flood extent
# The 'flooded' band in this collection is typically 1 for flooded, 0 otherwise
flood_viz = {
  'min': 0,
  'max': 1,
  'palette': ['white', 'blue'] # White for non-flooded, blue for flooded
}

# Create a geemap.Map object
# Use the centroid of South Sulawesi for centering
center = south_sulawesi.geometry().centroid().coordinates().getInfo()
Map = geemap.Map(center=center[::-1], zoom=7.5)

# Add the flood layer to the map
if first_flood_event:
    Map.addLayer(first_flood_event.select('flooded').clip(south_sulawesi), flood_viz, 'MODIS Flood Event')
    Map.addLayer(south_sulawesi, {'color': 'red', 'fillColor': '00000000'}, 'South Sulawesi Boundary')
else:
    print("No flood events found in the collection for this region.")

# Display the map
display(Map)

Map(center=[-3.7190754116758753, 120.17877389060263], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
def extract_features(image):
    """
    Extracts features from a single flood event image.
    """
    event_date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')

    # Get the bands to extract 
    bands_to_extract = image.bandNames()

    # Sample pixels from the image within the region
    samples = image.select(bands_to_extract).sample(
        region=south_sulawesi,
        scale=250,
        geometries=True,
        tileScale=8
    )

    samples = samples.map(lambda f: f.set('date', event_date))

    return samples

all_flood_features = flood_events.map(extract_features).flatten()

# Export to Google Drive
task = ee.batch.Export.table.toDrive(
    collection=all_flood_features,
    description='modis_flood_features_sulsel_15_5_25_23_39',
    fileFormat='CSV'
)

task.start()

print("Export of all flood event features started. Check your Google Drive after completion.")

NameError: name 'flood_events' is not defined

In [7]:
df = pd.read_csv('modis_flood_features_sulsel_15_5_25_23_39.csv')
df.head()

,system:index,clear_perc,clear_views,date,duration,flooded,jrc_perm_water,.geo
0,DFO_2403_From_20031210_to_20040123_0,1.000000,79,2003-12-10,0,0,0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
1,DFO_2403_From_20031210_to_20040123_1,1.000000,78,2003-12-10,0,0,0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
2,DFO_2403_From_20031210_to_20040123_2,0.974359,76,2003-12-10,0,0,0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
3,DFO_2403_From_20031210_to_20040123_3,1.000000,79,2003-12-10,0,0,0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."
4,DFO_2403_From_20031210_to_20040123_4,0.987342,78,2003-12-10,0,0,0,"{""geodesic"":false,""type"":""Point"",""coordinates""..."


In [8]:
def get_coordinates(geo:str):
    try:
        # Parse the GeoJSON string
        geojson = json.loads(geo)
        # Extract coordinates
        coords = geojson['coordinates']
        lat, lon = tuple(coords)
        
        return lat, lon
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error parsing coordinates: {e}")
        return None, None
        

In [10]:
get_coordinates(df['.geo'].loc[0])
latlon = df['.geo'].apply(get_coordinates)


In [13]:
df["latlon"] = latlon

In [23]:
df.query("flooded == 1 and jrc_perm_water == 1")


,system:index,clear_perc,clear_views,date,duration,flooded,jrc_perm_water,.geo,latlon
13866,DFO_2403_From_20031210_to_20040123_13866,1.0,79,2003-12-10,1,1,1,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(121.25347415634786, -2.4288199494381564)"
13867,DFO_2403_From_20031210_to_20040123_13867,1.0,79,2003-12-10,4,1,1,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(121.26245730918906, -2.4288199494381564)"
13868,DFO_2403_From_20031210_to_20040123_13868,1.0,79,2003-12-10,2,1,1,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(121.26470309739936, -2.4288199494381564)"
13869,DFO_2403_From_20031210_to_20040123_13869,1.0,79,2003-12-10,2,1,1,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(121.26694888560965, -2.4288199494381564)"
13870,DFO_2403_From_20031210_to_20040123_13870,1.0,79,2003-12-10,5,1,1,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(121.27144046203026, -2.4288199494381564)"
...,...,...,...,...,...,...,...,...,...
2172127,DFO_4222_From_20150205_to_20150209_310249,1.0,15,2015-02-05,1,1,1,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.59832824535764, -5.617839208062457)"
2172128,DFO_4222_From_20150205_to_20150209_310250,1.0,15,2015-02-05,1,1,1,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.60057403356794, -5.617839208062457)"
2172151,DFO_4222_From_20150205_to_20150209_310273,1.0,16,2015-02-05,3,1,1,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.54218354010017, -5.620084996272756)"
2172170,DFO_4222_From_20150205_to_20150209_310292,1.0,15,2015-02-05,1,1,1,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.59608245714733, -5.620084996272756)"


In [24]:
df.query("flooded == 1 and jrc_perm_water == 0")


,system:index,clear_perc,clear_views,date,duration,flooded,jrc_perm_water,.geo,latlon
17,DFO_2403_From_20031210_to_20040123_17,0.987179,77,2003-12-10,1,1,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(120.0632064048895, -1.979662307378395)"
124,DFO_2403_From_20031210_to_20040123_124,0.974684,77,2003-12-10,2,1,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.99807854679084, -2.0492817418976585)"
726,DFO_2403_From_20031210_to_20040123_726,1.000000,79,2003-12-10,1,1,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.85434810133171, -2.038052800846164)"
867,DFO_2403_From_20031210_to_20040123_867,0.962025,76,2003-12-10,1,1,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.88578913627589, -2.098689082524232)"
1462,DFO_2403_From_20031210_to_20040123_1462,0.986842,75,2003-12-10,1,1,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(120.46744828274328, -1.9774165191680968)"
...,...,...,...,...,...,...,...,...,...
2171959,DFO_4222_From_20150205_to_20150209_310081,1.000000,16,2015-02-05,1,1,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.56688721041346, -5.611101843431562)"
2172010,DFO_4222_From_20150205_to_20150209_310132,1.000000,16,2015-02-05,1,1,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.56913299862376, -5.613347631641859)"
2172061,DFO_4222_From_20150205_to_20150209_310183,1.000000,16,2015-02-05,1,1,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.56688721041346, -5.615593419852158)"
2172062,DFO_4222_From_20150205_to_20150209_310184,1.000000,16,2015-02-05,1,1,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.56913299862376, -5.615593419852158)"


In [26]:
df.query("flooded == 0 and jrc_perm_water == 1")


,system:index,clear_perc,clear_views,date,duration,flooded,jrc_perm_water,.geo,latlon
4336,DFO_2403_From_20031210_to_20040123_4336,1.000000,79,2003-12-10,0,0,1,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.65447295061512, -2.323267903554112)"
7334,DFO_2403_From_20031210_to_20040123_7334,0.974684,77,2003-12-10,0,0,1,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.8768059834347, -2.2738605629275384)"
7483,DFO_2403_From_20031210_to_20040123_7483,0.987342,78,2003-12-10,0,0,1,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.8835433480656, -2.282843715768734)"
7952,DFO_2403_From_20031210_to_20040123_7952,0.974684,77,2003-12-10,0,0,1,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.72858396155497, -2.323267903554112)"
12176,DFO_2403_From_20031210_to_20040123_12176,0.974684,77,2003-12-10,0,0,1,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(120.81779124354989, -2.4310657376484546)"
...,...,...,...,...,...,...,...,...,...
2172129,DFO_4222_From_20150205_to_20150209_310251,1.000000,15,2015-02-05,0,0,1,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.61180297461944, -5.617839208062457)"
2172130,DFO_4222_From_20150205_to_20150209_310252,1.000000,15,2015-02-05,0,0,1,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.61404876282973, -5.617839208062457)"
2172131,DFO_4222_From_20150205_to_20150209_310253,1.000000,15,2015-02-05,0,0,1,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.61629455104003, -5.617839208062457)"
2172152,DFO_4222_From_20150205_to_20150209_310274,1.000000,16,2015-02-05,0,0,1,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.54442932831047, -5.620084996272756)"


In [25]:
df.query("flooded == 0 and jrc_perm_water == 0")


,system:index,clear_perc,clear_views,date,duration,flooded,jrc_perm_water,.geo,latlon
0,DFO_2403_From_20031210_to_20040123_0,1.000000,79,2003-12-10,0,0,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.99134118215994, -1.9661875781166023)"
1,DFO_2403_From_20031210_to_20040123_1,1.000000,78,2003-12-10,0,0,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(120.08117271057189, -1.9684333663269014)"
2,DFO_2403_From_20031210_to_20040123_2,0.974359,76,2003-12-10,0,0,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(120.08791007520279, -1.9684333663269014)"
3,DFO_2403_From_20031210_to_20040123_3,1.000000,79,2003-12-10,0,0,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(120.01604485247323, -1.9729249427474986)"
4,DFO_2403_From_20031210_to_20040123_4,0.987342,78,2003-12-10,0,0,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(120.03401115815561, -1.9729249427474986)"
...,...,...,...,...,...,...,...,...,...
2172186,DFO_4222_From_20150205_to_20150209_310308,1.000000,14,2015-02-05,0,0,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.64998137419451, -5.620084996272756)"
2172187,DFO_4222_From_20150205_to_20150209_310309,1.000000,15,2015-02-05,0,0,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.65222716240481, -5.620084996272756)"
2172188,DFO_4222_From_20150205_to_20150209_310310,1.000000,15,2015-02-05,0,0,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.6567187388254, -5.620084996272756)"
2172189,DFO_4222_From_20150205_to_20150209_310311,1.000000,15,2015-02-05,0,0,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.6589645270357, -5.620084996272756)"


In [21]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df.head()

,system:index,clear_perc,clear_views,date,duration,flooded,jrc_perm_water,.geo,latlon
0,DFO_2403_From_20031210_to_20040123_0,1.000000,79,2003-12-10,0,0,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(119.99134118215994, -1.9661875781166023)"
1,DFO_2403_From_20031210_to_20040123_1,1.000000,78,2003-12-10,0,0,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(120.08117271057189, -1.9684333663269014)"
2,DFO_2403_From_20031210_to_20040123_2,0.974359,76,2003-12-10,0,0,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(120.08791007520279, -1.9684333663269014)"
3,DFO_2403_From_20031210_to_20040123_3,1.000000,79,2003-12-10,0,0,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(120.01604485247323, -1.9729249427474986)"
4,DFO_2403_From_20031210_to_20040123_4,0.987342,78,2003-12-10,0,0,0,"{""geodesic"":false,""type"":""Point"",""coordinates""...","(120.03401115815561, -1.9729249427474986)"


In [27]:
df.drop(columns=['.geo'], inplace=True)

In [31]:
df[['lat', 'lon']] = pd.DataFrame(df['latlon'].tolist(), index=df.index)
df.drop(columns=['latlon'], inplace=True)

In [32]:
df.to_csv('modis_flood_features_sulsel_15_5_25_23_39_cleaned.csv', index=False)